In [ ]:
! pip install -q --upgrade google-generativeai langchain-google-genai chromadb pypdf langchain langchain-community pysqlite3-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00


In [ ]:
from IPython.display import display
from IPython.display import Markdown
import google.generativeai as genai
import urllib
import warnings
import pandas as pd
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from pathlib import Path as p
from pprint import pprint
import textwrap
import warnings
import sqlite3

warnings.filterwarnings("ignore")

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
GOOGLE_API_KEY= "YOUR_API_KEY"
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=GOOGLE_API_KEY,
                             temperature=0.2,convert_system_message_to_human=True)

In [ ]:
file_contents = """
[
    {
        "start": 0.289,
        "end": 7.792,
        "text": "4 comes after 2"
    },
    {
        "start": 8.033,
        "end": 17.777,
        "text": "The Tesla Bot showcased its remarkable capabilities by performing what might seem like a simple task to humans but represents an extraordinary achievement in robotics, catching a tennis ball in mid-air."
    },
    {
        "start": 17.937,
        "end": 26.141,
        "text": "What makes this demonstration particularly jaw-dropping is that the Tesla Bot didn't just succeed once, it managed to snatch the tennis ball out of the air twice in succession."
    },
    {
        "start": 26.261,
        "end": 28.722,
        "text": "If you're watching this footage and thinking it must be CGI,"
    },
    {
        "start": 29.042,
        "end": 29.662,
        "text": " you're not alone."
    },
    {
        "start": 29.823,
        "end": 36.908,
        "text": "The fluid motion and precise timing appear almost too perfect to be real, but what we're witnessing is genuine robotics technology in action."
    },
    {
        "start": 37.368,
        "end": 41.291,
        "text": "The demonstration highlights two crucial aspects of Tesla's robotic development."
    },
    {
        "start": 41.391,
        "end": 48.516,
        "text": "First, the incredible speed of the robot's response time, allowing it to track and intercept a fast-moving object in three-dimensional space."
    },
    {
        "start": 48.817,
        "end": 50.378,
        "text": "Second, and perhaps even more impressively,"
    },
    {
        "start": 50.738,
        "end": 60.403,
        "text": " the sophisticated dexterity of its hands, which can delicately yet firmly grasp a small object moving through space, a task that requires precise finger coordination and pressure control."
    },
    {
        "start": 60.603,
        "end": 65.445,
        "text": "Now, in the interest of full transparency, I should mention that this demonstration was teleoperated."
    },
    {
        "start": 65.845,
        "end": 68.766,
        "text": " meaning a human operator was controlling the robot's movements."
    },
    {
        "start": 69.106,
        "end": 73.068,
        "text": "However, this shouldn't detract from the technological achievement we're witnessing."
    },
    {
        "start": 73.408,
        "end": 81.291,
        "text": "The ability to translate human movements into such precise robotic actions in real time is itself a remarkable feat of engineering."
    },
    {
        "start": 81.531,
        "end": 85.553,
        "text": "What's truly exciting is that this is just a glimpse of what's possible."
    },
    {
        "start": 85.933,
        "end": 91.395,
        "text": "Tesla's robotics platform is rapidly evolving and the rate of improvement we're seeing is nothing short of extraordinary."
    },
    {
        "start": 91.835,
        "end": 101.619,
        "text": " As the technology continues to mature, we can expect to see even more impressive capabilities emerge, potentially leading us toward a future where such robots can perform these kinds of tasks autonomously."
    },
    {
        "start": 102.039,
        "end": 110.042,
        "text": "In addition to the impressive ball-catching demonstration, we received another fascinating glimpse into Tesla's progress through a tweet shared by Sawyer Merritt."
    },
    {
        "start": 110.202,
        "end": 119.746,
        "text": "The tweet showcased an up-close look at Tesla's next generation robotic hand, which features an incredible 22 degrees of freedom approaching the complexity of the human hand itself."
    },
    {
        "start": 119.986,
        "end": 126.631,
        "text": " This detailed footage captured at the Wii Robot event reveals just how far Tesla has pushed the boundaries of robotic hand design."
    },
    {
        "start": 126.691,
        "end": 135.377,
        "text": "The hand's movements are remarkably fluid and natural, capable of executing a variety of intricate poses that mirror human hand movements with uncanny precision."
    },
    {
        "start": 135.837,
        "end": 139.28,
        "text": "Each joint and finger operates with a level of sophistication."
    },
    {
        "start": 139.56,
        "end": 145.464,
        "text": " that demonstrates why this isn't just another robotic gripper it's a significant leap forward in biomimetic engineering."
    },
    {
        "start": 145.664,
        "end": 155.23,
        "text": "The 22 degrees of freedom are particularly noteworthy as this level of articulation allows the hand to perform complex manipulations that were previously impossible for robotic systems."
    },
    {
        "start": 155.41,
        "end": 163.635,
        "text": " Watching the hand move through various positions, you can observe the subtle interplay between joints and actuators that make such sophisticated movement possible."
    },
    {
        "start": 163.775,
        "end": 165.937,
        "text": "This isn't just about mimicking human hand positions."
    },
    {
        "start": 166.237,
        "end": 172.14,
        "text": "It's about creating a versatile tool that could potentially match or even exceed human dexterity in certain tasks."
    },
    {
        "start": 172.421,
        "end": 178.244,
        "text": "Speaking of future capabilities, Elon Musk himself has painted an ambitious picture of what's to come for the Tesla bot."
    },
    {
        "start": 178.324,
        "end": 180.225,
        "text": "In a recent tweet, he made a remarkable claim."
    },
    {
        "start": 180.586,
        "end": 184.188,
        "text": "The robot hand will eventually be capable of playing both piano and guitar."
    },
    {
        "start": 184.488,
        "end": 188.91,
        "text": " While this might sound like science fiction, it's worth considering the implications of such a development."
    },
    {
        "start": 189.09,
        "end": 194.992,
        "text": "To put this in perspective, we haven't yet seen any robotic system successfully master these complex musical instruments."
    },
    {
        "start": 195.072,
        "end": 203.915,
        "text": "Playing piano or guitar requires not just precise finger placement but also careful control of pressure, timing and subtle movements that even many humans struggle to master."
    },
    {
        "start": 204.136,
        "end": 210.118,
        "text": "Each note on a guitar, for instance, needs exactly the right amount of pressure on the fret and a perfectly timed pluck of the string."
    },
    {
        "start": 210.498,
        "end": 217.643,
        "text": " Piano playing demands similar precision with the added complexity of managing multiple keys simultaneously while controlling the dynamics of each note."
    },
    {
        "start": 217.883,
        "end": 222.386,
        "text": "What makes Musk's claim particularly intriguing is his track record with the Tesla Bot project."
    },
    {
        "start": 222.466,
        "end": 227.949,
        "text": "While he's known for optimistic timelines in various ventures, the robot's rapid development suggests this might be different."
    },
    {
        "start": 228.029,
        "end": 235.234,
        "text": "We've already witnessed the hand performing remarkably complex tasks from the tennis ball catches to the intricate finger movements demonstrated at WeRobot."
    },
    {
        "start": 235.274,
        "end": 239.937,
        "text": "These achievements lend credibility to the possibility of more sophisticated operations in the near future."
    },
    {
        "start": 240.197,
        "end": 248.243,
        "text": " And this is where we get Elon Musk on Lex Fridman actually going into the details of the remarkable level of engineering it takes to actually build such a robot hand."
    },
    {
        "start": 248.723,
        "end": 251.505,
        "text": "Do things that a human can do, it's actually a very high bar."
    },
    {
        "start": 252.186,
        "end": 258.57,
        "text": "So our new arm has 22 degrees of freedom instead of 11 and has the actuators in the forearm."
    },
    {
        "start": 259.851,
        "end": 267.797,
        "text": "And all the actuators are designed from scratch, the physics first principles, but the sensors are all designed from scratch."
    },
    {
        "start": 268.377,
        "end": 280.273,
        "text": " And we'll continue to put a tremendous amount of engineering effort into improving the hand, like the hand, but by hand, I mean like the entire forearm from elbow forward is really the hand."
    },
    {
        "start": 281.926,
        "end": 287.671,
        "text": " Now it actually gets even more interesting because Elon Musk discusses something that I didn't even think was possible."
    },
    {
        "start": 287.791,
        "end": 293.876,
        "text": "Take a look at another clip from a year earlier where he discussed having the Tesla bot be able to thread a needle just by vision alone."
    },
    {
        "start": 302.739,
        "end": 302.959,
        "text": " You know?"
    },
    {
        "start": 303.64,
        "end": 303.86,
        "text": "Yeah."
    },
    {
        "start": 304.861,
        "end": 309.005,
        "text": "And I could do some interesting manipulation, soft, soft touch robotics."
    },
    {
        "start": 309.725,
        "end": 322.977,
        "text": "I mean, one of the tests goals I have is can, can it pick up a needle and a thread and thread the needle just by looking how far away away from that just by looking just by looking, maybe a year."
    },
    {
        "start": 324.144,
        "end": 326.205,
        "text": " This might seem almost impossible at first glance."
    },
    {
        "start": 326.425,
        "end": 329.867,
        "text": "Threading a needle combines challenges at both the macro and micro scales."
    },
    {
        "start": 330.147,
        "end": 336.67,
        "text": "You need broad arm movements to position the thread, but then incredible precision to hit a target that's often just a fraction of a millimeter wide."
    },
    {
        "start": 337.03,
        "end": 341.092,
        "text": "What makes Musk's vision particularly ambitious is the vision alone aspect."
    },
    {
        "start": 341.232,
        "end": 348.994,
        "text": " no special guides, no mechanical assistance, just the robot's visual system and its remarkably dexterous hand working together to accomplish this intricate task."
    },
    {
        "start": 349.375,
        "end": 355.776,
        "text": "When you consider the precision required, even the slightest tremor or miscalculation of a fraction of a millimeter means failure."
    },
    {
        "start": 355.957,
        "end": 362.118,
        "text": "It's not just impressive, it's a statement about Tesla's confidence in their robotic vision systems and fine motor control."
    },
    {
        "start": 362.218,
        "end": 370.281,
        "text": "It's one thing to catch a tennis ball in midair, which requires quick reflexes and spatial awareness, but threading a needle demands a whole different level of precision and control."
    },
    {
        "start": 370.561,
        "end": 374.584,
        "text": " If Tesla can achieve this goal, it would represent a quantum leap in robotic capabilities."
    },
    {
        "start": 374.924,
        "end": 383.29,
        "text": "We'd be looking at a system that can seamlessly transition between dynamic, fast-paced tasks like ball catching and ultra-precise operations at near-microscopic scales."
    },
    {
        "start": 383.41,
        "end": 389.174,
        "text": "This kind of versatility is precisely what would make humanoid robots truly useful in real-world applications."
    },
    {
        "start": 389.735,
        "end": 394.938,
        "text": "I mean, like your hands, the actuators, the muscles of your hand are almost overwhelmingly in your forearm."
    },
    {
        "start": 396.334,
        "end": 400.177,
        "text": " So your forearm has the muscles that actually control your hand."
    },
    {
        "start": 401.397,
        "end": 409.523,
        "text": "There's a few small muscles in the hand itself, but your hand is really like a skeleton meat puppet and with cables."
    },
    {
        "start": 410.644,
        "end": 423.652,
        "text": "So the muscles that control your fingers are in your forearm and they go through the carpal tunnel, which is that you've got a little collection of bones and a tiny tunnel that these cables, the tendons go through."
    },
    {
        "start": 424.093,
        "end": 425.574,
        "text": "And those tendons are"
    },
    {
        "start": 426.614,
        "end": 428.697,
        "text": " what, um, mostly what moves your hands."
    },
    {
        "start": 429.939,
        "end": 435.305,
        "text": "And something like those tenants has to be re-engineered into the optimus and do all that kind of stuff."
    },
    {
        "start": 435.706,
        "end": 435.866,
        "text": "Yeah."
    },
    {
        "start": 435.886,
        "end": 440.091,
        "text": "So like the current optimus, um, we tried putting the actuators in the hand itself."
    },
    {
        "start": 440.972,
        "end": 445.017,
        "text": "Then you sort of end up having these like giant hands, giant hands that look weird."
    },
    {
        "start": 445.997,
        "end": 450.098,
        "text": " And then they don't actually have enough degrees of freedom or enough strength."
    },
    {
        "start": 451.079,
        "end": 455.06,
        "text": "So then you realize, okay, that's why you've got to put the actuators in the forearm."
    },
    {
        "start": 455.88,
        "end": 463.062,
        "text": "And just like a human, you've got to run cables through a narrow tunnel to operate the fingers."
    },
    {
        "start": 463.722,
        "end": 467.663,
        "text": "And then there's also a reason for not having all the fingers the same length."
    },
    {
        "start": 468.383,
        "end": 471.904,
        "text": "So it wouldn't be expensive from an energy or evolutionary standpoint to have all your fingers be the same length."
    },
    {
        "start": 471.924,
        "end": 472.844,
        "text": "So why not do the same length?"
    },
    {
        "start": 473.223,
        "end": 473.783,
        "text": " Yeah, why not?"
    },
    {
        "start": 474.244,
        "end": 475.625,
        "text": "Because it's actually better to have different lengths."
    },
    {
        "start": 476.465,
        "end": 478.706,
        "text": "Your dexterity is better if you've got fingers a different length."
    },
    {
        "start": 481.228,
        "end": 487.112,
        "text": "There are more things you can do and your dexterity is actually better if your fingers are a different length."
    },
    {
        "start": 488.372,
        "end": 489.633,
        "text": "Like there's a reason we've got a little finger."
    },
    {
        "start": 489.653,
        "end": 490.774,
        "text": "Why not have a little finger that's bigger?"
    },
    {
        "start": 491.594,
        "end": 491.855,
        "text": "Yeah."
    },
    {
        "start": 491.935,
        "end": 493.836,
        "text": "Because it allows you to do fine."
    },
    {
        "start": 493.856,
        "end": 495.277,
        "text": "It helps you with fine motor skills."
    },
    {
        "start": 496.294,
        "end": 497.596,
        "text": " That this little finger helps."
    },
    {
        "start": 497.876,
        "end": 498.176,
        "text": "It does."
    },
    {
        "start": 498.797,
        "end": 498.957,
        "text": "Hmm."
    },
    {
        "start": 501.0,
        "end": 505.065,
        "text": "If you lost your little finger, it would have noticeably less dexterity."
    },
    {
        "start": 505.512,
        "end": 512.679,
        "text": " What's particularly exciting is that these aren't just technical demonstrations or laboratory experiments, we're seeing the emergence of practical, versatile robotics that could fundamentally transform"
    },
    {
        "start": 528.553,
        "end": 529.834,
        "text": " how we live and work."
    },
    {
        "start": 529.914,
        "end": 537.198,
        "text": "4 comes before 6."
    },
    {
        "start": 537.299,
        "end": 547.385,
        "text": "While it's impossible to predict exactly what life will look like in 2034, one thing is certain, the robotics revolution is not just coming, it's already here and it's accelerating faster than many of us ever anticipated."
    }
]
"""

# Now file_contents contains the text as a string
print(file_contents)


[
    {
        "start": 0.289,
        "end": 7.792,
        "text": "4 comes after 2"
    },
    {
        "start": 8.033,
        "end": 17.777,
        "text": "The Tesla Bot showcased its remarkable capabilities by performing what might seem like a simple task to humans but represents an extraordinary achievement in robotics, catching a tennis ball in mid-air."
    },
    {
        "start": 17.937,
        "end": 26.141,
        "text": "What makes this demonstration particularly jaw-dropping is that the Tesla Bot didn't just succeed once, it managed to snatch the tennis ball out of the air twice in succession."
    },
    {
        "start": 26.261,
        "end": 28.722,
        "text": "If you're watching this footage and thinking it must be CGI,"
    },
    {
        "start": 29.042,
        "end": 29.662,
        "text": " you're not alone."
    },
    {
        "start": 29.823,
        "end": 36.908,
        "text": "The fluid motion and precise timing appear almost too perf

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
context = file_contents
texts = text_splitter.split_text(context)

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=GOOGLE_API_KEY)

In [ ]:
__import__('pysqlite3')
import sys

sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

<module 'sqlite3' from '/usr/local/python/3.12.1/lib/python3.12/sqlite3/__init__.py'>

In [ ]:
vector_index = Chroma.from_texts(texts, embeddings).as_retriever(search_kwargs={"k":5})

In [ ]:
template = """Use the following json having description, timestamps with start and end point return only the timestamps and answer for correct answer only. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)# Run chain
qa_chain = RetrievalQA.from_chain_type(
    model,
    retriever=vector_index,
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)


In [ ]:
question = "What is after 2 and before 6?"
result = qa_chain({"query": question})
result["result"]

In [ ]:
temp=result["result"]

In [ ]:
import json
data = json.loads(temp)


In [ ]:
for item in data:
    print(f"{item['start']} - {item['end']}")

0.289 7.792
529.914 537.198


In [ ]:
del vector_index